In [ ]:
#importing libraries
!pip install -U datasets tiktoken tqdm numpy torch matplotlib huggingface_hub
import os
import math
import time
from contextlib import nullcontext
from dataclasses import dataclass, field
from tqdm.auto import tqdm
import json
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR
import tiktoken
from datasets import load_dataset
import matplotlib.pyplot as plt

In [ ]:
#tokenization implementation
from datasets import load_dataset
import numpy as np
from tqdm.auto import tqdm
import os
import tiktoken 
import math


enc = tiktoken.get_encoding("gpt2")
eot_token = enc.eot_token

In [ ]:
def processing_bpe(sample):
    text_data = sample.get('text', '')
    if not isinstance(text_data, str): text_data = ""
    ids = enc.encode_ordinary(text_data)
    ids.append(eot_token)
    out = {'ids': ids, 'len': len(ids)}
    return out
combined_data_dir = config.data_dir
os.makedirs(combined_data_dir, exist_ok=True)
train_filename = os.path.join(combined_data_dir, 'train_combined.bin')
val_filename = os.path.join(combined_data_dir, 'val_combined.bin')

In [ ]:
#tokenizer with data splits..
if os.path.exists(train_filename) and os.path.exists(val_filename):
    print(f"Combined .bin files already exist in {combined_data_dir}. Skipping processing.")

else:
    ts_df = load_dataset("roneneldan/TinyStories")
    bc_df = load_dataset("rojagtap/bookcorpus", split='train')
    
    ts_split = ts_df['train'].train_test_split(test_size=0.01, seed=42)
    ts_train_df = ts_split['train']
    ts_val_df = ts_split['test']
    
    bc_split = bc_df.train_test_split(test_size=0.01, seed=42)
    bc_train_df = bc_split['train']
    bc_val_df = bc_split['test']

    dataset_splits = {
        'train': {'ts': ts_train_df, 'bc': bc_train_df},
        'validation': {'ts': ts_val_df, 'bc': bc_val_df}
    }

    for split in ['train', 'validation']:
        filename = train_filename if split == 'train' else val_filename
        
        if os.path.exists(filename):
            os.remove(filename)

        tokenized_ts = dataset_splits[split]['ts'].map(
            processing_bpe, remove_columns=['text'],
            desc=f"Tokenizing TinyStories {split}", num_proc=config.num_proc
        )
        
        tokenized_bc = dataset_splits[split]['bc'].map(
            processing_bpe, remove_columns=['text'],
            desc=f"Tokenizing BookCorpus {split}", num_proc=config.num_proc
        )

        ts_len = np.sum(tokenized_ts['len'], dtype=np.uint64)
        bc_len = np.sum(tokenized_bc['len'], dtype=np.uint64)
        arr_len = ts_len + bc_len

        if arr_len == 0:
            print(f"Warning: No tokens found for combined {split} split. Skipping."); continue
            
        print(f"Total tokens for {split}: {arr_len:,} (TinyStories: {ts_len:,}, BookCorpus: {bc_len:,})")
        dtype_np = np.uint16
        arr = np.memmap(filename, dtype=dtype_np, mode='w+', shape=(arr_len,))
        
        print(f"Writing {ts_len:,} TinyStories tokens to {filename}...")
        idx = 0
        write_batch_size = 1000 # Docs per chunk
        
        total_samples_ts = len(tokenized_ts)
        num_write_batches_ts = math.ceil(total_samples_ts / write_batch_size)
        for i in tqdm(range(num_write_batches_ts), desc=f"Writing TinyStories {split}"):
            start = i * write_batch_size; end = min((i + 1) * write_batch_size, total_samples_ts)
            chunk = tokenized_ts.select(range(start, end))
            try:
                all_ids_in_chunk = [item['ids'] for item in chunk if item['ids']]
                if not all_ids_in_chunk: continue
                arr_batch = np.concatenate(all_ids_in_chunk).astype(dtype_np)
                batch_len = len(arr_batch)
                expected_end_idx = idx + batch_len
                if expected_end_idx > arr_len: print(f"Error: Bounds overflow (TS). Stopping."); break
                arr[idx : expected_end_idx] = arr_batch; idx = expected_end_idx
            except Exception as e: print(f"Error writing TS chunk {i}: {e}"); break
        
        
        total_samples_bc = len(tokenized_bc)
        num_write_batches_bc = math.ceil(total_samples_bc / write_batch_size)
        
        for i in tqdm(range(num_write_batches_bc), desc=f"Writing BookCorpus {split}"):
            start = i * write_batch_size; end = min((i + 1) * write_batch_size, total_samples_bc)
            chunk = tokenized_bc.select(range(start, end))
            try:
                all_ids_in_chunk = [item['ids'] for item in chunk if item['ids']]
                if not all_ids_in_chunk: continue
                arr_batch = np.concatenate(all_ids_in_chunk).astype(dtype_np)
                batch_len = len(arr_batch)
                expected_end_idx = idx + batch_len
                if expected_end_idx > arr_len: print(f"Error: Bounds overflow (BC). Stopping."); break
                arr[idx : expected_end_idx] = arr_batch; idx = expected_end_idx
            except Exception as e: print(f"Error writing BC chunk {i}: {e}"); break

        arr.flush()
        if idx != arr_len:
             print(f"Warning: Final index {idx} doesn't match expected {arr_len}.")
        else:
            print(f"Successfully wrote {idx} total tokens.")
        print(f"Finished writing {filename}.")

In [ ]:
#memap files
import numpy as np
import torch
import os
train_data_file = os.path.join(config.data_dir, 'train_combined.bin')
val_data_file = os.path.join(config.data_dir, 'val_combined.bin')
if not os.path.exists(train_data_file) or not os.path.exists(val_data_file):
     raise FileNotFoundError(f"Combined .bin files not found in {config.data_dir}. Did Cell 3 complete?")

train_data = np.memmap(train_data_file, dtype=np.uint16, mode='r')
val_data = np.memmap(val_data_file, dtype=np.uint16, mode='r')

print(f"Train data loaded: {len(train_data):,} tokens")
print(f"Validation data loaded: {len(val_data):,} tokens")